#### Classify new occurences of audio
Very naive approach.
From: https://www.youtube.com/watch?v=VMXTSavBrJo&t

In [ ]:
# !conda create -n hcktn python=3.12 pip jupyterlab
# !pip install jupyterlab-sublime

In [3]:
!pip install librosa scipy matplotlib

  Using cached matplotlib-3.9.2-cp312-cp312-macosx_10_12_x86_64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp312-cp312-macosx_10_13_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.0-cp312-cp312-macosx_10_13_x86_64.whl.metadata (164 kB)
  Using cached kiwisolver-1.4.7-cp312-cp312-macosx_10_9_x86_64.whl.metadata (6.3 kB)
  Using cached pillow-11.0.0-cp312-cp312-macosx_10_13_x86_64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.9.2-cp312-cp312-macosx_10_12_x86_64.whl (7.9 MB)
Using cached contourpy-1.3.1-cp312-cp312-macosx_10_13_x86_64.whl (271 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.55.0-cp312-cp312-macosx_10_13_x86_64.whl (2.3 MB)
Using cached kiwisolver-1.4.7-cp312-cp312-macosx_10_9_x86_64.whl (65 kB)
Using cached pillow-11.0.0-cp312-cp312-macosx_10_13_x86_64.whl (3.1 MB)
Using cached pyparsing-3

In [3]:
!conda install ffmpeg -y

Solving environment: done

## Package Plan ##

  environment location: /usr/local/Caskroom/miniconda/base/envs/taildropout

  added / updated specs:
    - ffmpeg


The following NEW packages will be INSTALLED:

  aom                pkgs/main/osx-64::aom-3.6.0-hcec6c5f_0 
  cairo              pkgs/main/osx-64::cairo-1.16.0-h3ce6f7e_5 
  dav1d              pkgs/main/osx-64::dav1d-1.2.1-h6c40b1e_0 
  ffmpeg             pkgs/main/osx-64::ffmpeg-6.1.1-h931e7ea_0 
  fontconfig         pkgs/main/osx-64::fontconfig-2.14.1-hb0a0c50_2 
  freetype           pkgs/main/osx-64::freetype-2.12.1-hd8bbffd_0 
  fribidi            pkgs/main/osx-64::fribidi-1.0.10-haf1e3a3_0 
  gdk-pixbuf         pkgs/main/osx-64::gdk-pixbuf-2.42.10-h46256e1_1 
  gettext            pkgs/main/osx-64::gettext-0.21.0-he85b6c0_1 
  giflib             pkgs/main/osx-64::giflib-5.2.2-h46256e1_0 
  glib               pkgs/main/osx-64::glib-2.78.4-hcec6c5f_0 
  glib-tools         pkgs/main/osx-64::glib-tools-2.78.4-hcec6c5f_0 
  g

In [4]:
import librosa
import numpy as np
from scipy.spatial.distance import cdist
from IPython.display import Audio
import matplotlib.pyplot as plt


In [5]:
# Function to load an audio file
def load_audio(file_path, sr=22050):
    audio, _ = librosa.load(file_path, sr=sr)
    return audio, sr

# Hardcoded timestamps and descriptions (in seconds)
timestamps = [
    (0, "Casual Morning Artillery Exchange"),
    (12 * 60 + 5, "Distant Gunfire"),
    (33 * 60 + 45, "Shelling Has Started Nearby"),
    (39 * 60 + 8, "Shelling Continues"),
    (41 * 60 + 14, "An FPV Drone Flies by"),
    (42 * 60 + 41, "Mavic is Flying Nearby"),
    (55 * 60 + 9, "Allied Gun Nearby Starts Shooting"),
    (69 * 60 + 15, "Allied Gun Shoots Again with Different Shells"),
    (71 * 60 + 24, "Another FPV Drone Buzzing Around"),
    (103 * 60 + 48, "Heavy Machine Gun Shooting Somewhere"),
    (109 * 60 + 23, "Enemy Tank Starts Shooting Our Way"),
    (123 * 60 + 33, "Nightingale Song"),
]

# Load your full audio file (MP3 format)
full_audio_path = "/Users/gunjack/Downloads/Summer in Trenches _ Real War Ambient _ Rus-Ukrainian Frontline Background Sounds.mp3"  # Replace with your actual file path
full_audio, sr = load_audio(full_audio_path)

# Display sample rate and audio duration
print(f"Sample rate: {sr} Hz")
print(f"Audio duration: {len(full_audio) / sr:.2f} seconds")


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?


Sample rate: 22050 Hz
Audio duration: 7520.27 seconds


In [6]:
# Function to extract 3-second clips around each timestamp
def extract_clips(audio, sr, timestamps, window_size=5.0):
    clips = []
    half_window = window_size / 2
    for time_sec, description in timestamps:
        start_time = max(0, time_sec - half_window)
        end_time = min(len(audio) / sr, time_sec + half_window)
        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)
        clip_audio = audio[start_sample:end_sample]
        clips.append({
            'audio': clip_audio,
            'description': description,
            'time_sec': time_sec,
            'start_time': start_time,
            'end_time': end_time
        })
    return clips

# Extract clips
clips = extract_clips(full_audio, sr, timestamps)

# Function to play a clip
def play_clip(clip_index):
    clip = clips[clip_index]
    print(f"Clip {clip_index}: {clip['description']} (around {clip['time_sec']} seconds)")
    return Audio(clip['audio'], rate=sr)

# Play the first clip interactively
clip_index = 2  # Change index to explore other clips (0 to 11)
play_clip(clip_index)


Clip 2: Shelling Has Started Nearby (around 2025 seconds)


In [7]:
# Function to extract MFCC features
def extract_mfcc(audio, sr, n_mfcc=13):
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return mfcc.T  # Transpose to have time-axis rows

# Function to find matches in the full audio
def find_matches(target_audio, full_audio, sr, segment_duration=3.0):
    # Extract MFCC for the target audio
    target_mfcc = extract_mfcc(target_audio, sr)
    target_mean = np.mean(target_mfcc, axis=0)

    # Prepare to iterate over the full audio
    segment_length = int(segment_duration * sr)
    hop_length = int(segment_length / 2)  # 50% overlap
    matches = []

    for start_sample in range(0, len(full_audio) - segment_length + 1, hop_length):
        segment = full_audio[start_sample:start_sample + segment_length]
        segment_mfcc = extract_mfcc(segment, sr)
        segment_mean = np.mean(segment_mfcc, axis=0)

        # Compute Euclidean distance between MFCC means
        distance = np.linalg.norm(target_mean - segment_mean)
        start_time = start_sample / sr
        end_time = (start_sample + segment_length) / sr
        matches.append({
            'start_time': start_time,
            'end_time': end_time,
            'distance': distance
        })

    # Sort matches by distance (lower is better)
    matches.sort(key=lambda x: x['distance'])
    return matches

# Run matching for the selected clip
selected_clip = clips[clip_index]['audio']
matches = find_matches(selected_clip, full_audio, sr)

# Function to display top matches
def display_top_matches(matches, top_k=5):
    print(f"Top {top_k} Matches for Clip {clip_index}:")
    for i in range(top_k):
        match = matches[i]
        print(f"{i+1}. Start: {match['start_time']:.2f}s, End: {match['end_time']:.2f}s, Distance: {match['distance']:.2f}")

display_top_matches(matches)

Top 5 Matches for Clip 2:
1. Start: 1885.50s, End: 1888.50s, Distance: 5.54
2. Start: 1903.50s, End: 1906.50s, Distance: 5.65
3. Start: 1717.50s, End: 1720.50s, Distance: 6.43
4. Start: 3033.00s, End: 3036.00s, Distance: 6.69
5. Start: 1984.50s, End: 1987.50s, Distance: 7.10


In [8]:
# Function to play a matching segment
def play_match(match_index):
    match = matches[match_index]
    start_sample = int(match['start_time'] * sr)
    end_sample = int(match['end_time'] * sr)
    match_audio = full_audio[start_sample:end_sample]
    print(f"Playing Match {match_index+1}: {match['start_time']:.2f}s to {match['end_time']:.2f}s")
    return Audio(match_audio, rate=sr)

display(play_clip(clip_index))
# Listen to the top match
display(play_match(match_index=0))
display(play_match(match_index=1))
display(play_match(match_index=3))
display(play_match(match_index=4))
display(play_match(match_index=5))

Clip 2: Shelling Has Started Nearby (around 2025 seconds)


Playing Match 1: 1885.50s to 1888.50s


Playing Match 2: 1903.50s to 1906.50s


Playing Match 4: 3033.00s to 3036.00s


Playing Match 5: 1984.50s to 1987.50s


Playing Match 6: 1923.00s to 1926.00s
